<a href="https://colab.research.google.com/github/hamednasr/TensorFlow-Projects/blob/main/MobileNetV2_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MobileNetv2 feature extraction transfer learning

### **mission** :we want to classify the images of ten different foods, using the MobileNetv2 and transfer learning, using only 10% of the training data which is only 75 photos in each class, the test set contains 250 images in each class. 

In [19]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import zipfile
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import datetime
from tensorflow.keras import layers, optimizers
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

##load data:

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-02-28 08:14:51--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.74.128, 209.85.145.128, 209.85.146.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.74.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   208MB/s    in 0.8s    

2022-02-28 08:14:52 (208 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

In [4]:
for dirpath, dirnames,filenames in os.walk('10_food_classes_10_percent'):
  print(f'there are {len(dirnames)} folders and {len(filenames)} images in {dirpath}')

there are 2 folders and 0 images in 10_food_classes_10_percent
there are 10 folders and 0 images in 10_food_classes_10_percent/test
there are 0 folders and 250 images in 10_food_classes_10_percent/test/fried_rice
there are 0 folders and 250 images in 10_food_classes_10_percent/test/chicken_wings
there are 0 folders and 250 images in 10_food_classes_10_percent/test/pizza
there are 0 folders and 250 images in 10_food_classes_10_percent/test/steak
there are 0 folders and 250 images in 10_food_classes_10_percent/test/grilled_salmon
there are 0 folders and 250 images in 10_food_classes_10_percent/test/sushi
there are 0 folders and 250 images in 10_food_classes_10_percent/test/chicken_curry
there are 0 folders and 250 images in 10_food_classes_10_percent/test/ramen
there are 0 folders and 250 images in 10_food_classes_10_percent/test/hamburger
there are 0 folders and 250 images in 10_food_classes_10_percent/test/ice_cream
there are 10 folders and 0 images in 10_food_classes_10_percent/train


## Preprocess data:

In [5]:
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

In [6]:
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

In [7]:
train_data = train_datagen.flow_from_directory(train_dir,target_size=(224,224))
test_data = test_datagen.flow_from_directory(test_dir,target_size=(224,224))

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


## building model 

In [8]:
MobileNetv2 = 'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5'
feature_Extractor_layers = hub.KerasLayer(MobileNetv2,name='MobileNetv2',
                                          trainable=False,input_shape= (224,224,3))

In [9]:
MobileNetmodel = tf.keras.Sequential([
                             
              feature_Extractor_layers,

              # layers.Dense(128,activation='relu'),
              layers.Dense(10,activation='softmax')
])

MobileNetmodel.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
                       optimizer = tf.keras.optimizers.Adam(),
                       metrics = ['accuracy'])

In [10]:
MobileNetmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobileNetv2 (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 10)                12810     
                                                                 
Total params: 2,270,794
Trainable params: 12,810
Non-trainable params: 2,257,984
_________________________________________________________________


## creat tensorboard callbacks:

In [11]:
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + '/' + experiment_name + '/' +datetime.datetime.now().strftime('%y%m%d-%H%M%S')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,write_images=True)
  print(f'saving tensorboard log files to: {log_dir}')
  return tensorboard_callback


## fit the model with 10% of the training data:

In [12]:
MobileNetmodel.fit(train_data, epochs=10, 
                   callbacks=create_tensorboard_callback('tensorboard', 'MobileNetv2'))

saving tensorboard log files to: tensorboard/MobileNetv2/220228-081502
Epoch 1/10
24/24 [==============================] - 18s 186ms/step - loss: 2.0499 - accuracy: 0.2773
Epoch 2/10
24/24 [==============================] - 4s 185ms/step - loss: 1.0643 - accuracy: 0.7067
Epoch 3/10
24/24 [==============================] - 4s 183ms/step - loss: 0.7517 - accuracy: 0.8013
Epoch 4/10
24/24 [==============================] - 4s 181ms/step - loss: 0.5845 - accuracy: 0.8427
Epoch 5/10
24/24 [==============================] - 4s 179ms/step - loss: 0.4816 - accuracy: 0.8813
Epoch 6/10
24/24 [==============================] - 4s 180ms/step - loss: 0.4065 - accuracy: 0.9160
Epoch 7/10
24/24 [==============================] - 4s 179ms/step - loss: 0.3473 - accuracy: 0.9400
Epoch 8/10
24/24 [==============================] - 4s 179ms/step - loss: 0.2988 - accuracy: 0.9533
Epoch 9/10
24/24 [==============================] - 4s 179ms/step - loss: 0.2680 - accuracy: 0.9680
Epoch 10/10
24/24 [=========

In [13]:
MobileNetmodel.evaluate(test_data)

79/79 [==============================] - 16s 189ms/step - loss: 0.5633 - accuracy: 0.8192


[0.5632665753364563, 0.8191999793052673]

In [21]:
y_pred = np.argmax(MobileNetmodel.predict(test_data),axis=1)
y_pred

array([5, 2, 8, ..., 8, 0, 8])